Endprojekt GIS Analyse - Location Analysis for a new Stadium in Graz

In [14]:
# Data manipulation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Geospatial
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon
import osmnx as ox

# Raster processing
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
import pyproj

import os
import pyproj
# optinal code fix with PROJ_LIB if pyproj doesnt work properly
os.environ["PROJ_LIB"] = pyproj.datadir.get_data_dir()


Data Preparation
===============================================================================     

Getting district boundaries of Graz (District boundaries (Overpass Turbo (n.d.). Overpass API web interface. https://overpass-turbo.eu/ (Accessed December 2, 2025))) <br>
and <br>
population statistic (Stadt Graz (2025). Zahlen + Fakten: Bevölkerung, Bezirke, Wirtschaft, Geografie. https://www.graz.at/cms/beitrag/10034466/7772565/Zahlen_Fakten_Bevoelkerung_Bezirke_Wirtschaft.html (Accessed December 2, 2025))

In [15]:
#loading GeoJson Districts of Graz
gdf_districts = gpd.read_file("data/district_graz.geojson")
gdf_districts = gdf_districts[["name", "geometry"]].copy() #extract the columns needed

#loading csv with pop data of graz 
df_popgraz = pd.read_csv("data/Bevoelkerung Graz.csv", encoding="latin1", sep=";")

#merge the two datasets
gdf_districts_popGraz = gdf_districts.merge(df_popgraz, left_on="name", right_on="Bezirk")

Getting hard exklusion layers from OpenStreetMap via OSMnx, set the CRS to 32633 (UTM zone 33N) and buffer some of them:<br>
- water bodies buffer (10m)<br>
- parks/green areas <br>
- transport infrastructure <br>
- buildings (buffer 250m for hospitals, kingardens, cemeteries)

In [16]:
PLACE_NAME:str = "Graz, Austria"
#TARGET_CRS = "EPSG:32633"  # UTM zone 33N

In [37]:
tags_water = {
    "natural": ["water"],
    "waterway": ["river", "stream", "canal", "ditch"],
    "landuse": ["reservoir"],
    "water": ["lake", "river", "pond", "basin"]
}
gdf_water = ox.features_from_place(
    PLACE_NAME,
    tags=tags_water
)
gdf_water = gdf_water[["geometry"]].copy()
gdf_water["category"] = "water"

# Reproject
gdf_water = gdf_water.to_crs(epsg=32633)

# 10 meter buffer
gdf_water_Buffer10m = gdf_water.copy()
gdf_water_Buffer10m["geometry"] = gdf_water_Buffer10m.buffer(10)
gdf_water_Buffer10m.head()

geometry category
element  id                                                                 
relation 1306807  POLYGON ((532504.7 5213425.005, 532504.197 521...    water
         2325656  POLYGON ((536875.338 5207617.465, 536875.714 5...    water
         3403202  POLYGON ((536217.762 5210905.412, 536219.526 5...    water
         3562966  POLYGON ((533585.365 5209757.598, 533585.028 5...    water
         3587716  POLYGON ((535239.839 5206807.914, 535240.715 5...    water

In [18]:
#Excklusion Parks
tags_parks = {
    "leisure": [
        "park", "garden", "playground", "recreation_ground"
    ],
    "boundary": [
        "protected_area"
    ]
}
gdf_parks = ox.features_from_place(
    PLACE_NAME,
    tags=tags_parks
)

gdf_parks = gdf_parks[["geometry"]].copy()
gdf_parks["category"] = "green areas"

# Reproject
gdf_parks = gdf_parks.to_crs(epsg=32633)

# 10 meter buffer
gdf_parks_Buffer10m = gdf_parks["geometry"] = gdf_parks.buffer(10)

In [19]:
#Exclusion Transport Areas
tags_transport = {
    "highway": [
        "motorway", "trunk", "primary", "secondary", "tertiary",
        "motorway_link", "trunk_link", "primary_link", "secondary_link"
    ],
    "railway": [
        "rail", "tram", "light_rail", "subway"
    ]
}

gdf_transport = ox.features_from_place(
    PLACE_NAME,
    tags=tags_transport
)

gdf_transport = gdf_transport[["geometry"]].copy()
gdf_transport["category"] = "transport infrastructure"

# Reproject
gdf_transport = gdf_transport.to_crs(epsg=32633)

# 5 meter buffer
gdf_transport_Buffer5m = gdf_transport["geometry"] = gdf_transport.buffer(5)

In [20]:
#Exclusion Buildings
tags_buildings_hard = {
    "building": True
}

gdf_buildings = ox.features_from_place(
    PLACE_NAME,
    tags=tags_buildings_hard
)

gdf_buildings = gdf_buildings[["geometry"]].copy()
gdf_buildings["category"] = "buildings_hard"

# Reproject
gdf_buildings = gdf_buildings.to_crs(epsg=32633)

In [21]:
#Kindergartens with 250m Buffer
tags_kindergarten = {
    "amenity": ["kindergarten"]
}

gdf_kindergarten = ox.features_from_place(
    PLACE_NAME,
    tags=tags_kindergarten
)

gdf_kindergarten = gdf_kindergarten[["geometry"]].copy()
gdf_kindergarten["category"] = "kindergarten"

gdf_kindergarten = gdf_kindergarten.to_crs(epsg=32633)
gdf_kindergarten["geometry"] = gdf_kindergarten.buffer(250)

gdf_kindergarten_Buffer_250m = gdf_kindergarten.copy()

In [22]:
# Hospitals with 250m Buffer
tags_hospital = {
    "amenity": ["hospital"]
}

gdf_hospital = ox.features_from_place(
    PLACE_NAME,
    tags=tags_hospital
)

gdf_hospital = gdf_hospital[["geometry"]].copy()
gdf_hospital["category"] = "hospital"

gdf_hospital = gdf_hospital.to_crs(epsg=32633)
gdf_hospital["geometry"] = gdf_hospital.buffer(250)

gdf_hospital_Buffer_250m = gdf_hospital.copy()

In [23]:
#Cemeteries with 250m Buffer
tags_cemetery = {
    "landuse": ["cemetery"],
    "amenity": ["grave_yard"]
}

gdf_cemetery = ox.features_from_place(
    PLACE_NAME,
    tags=tags_cemetery
)

gdf_cemetery = gdf_cemetery[["geometry"]].copy()
gdf_cemetery["category"] = "cemetery"

gdf_cemetery = gdf_cemetery.to_crs(epsg=32633)
gdf_cemetery["geometry"] = gdf_cemetery.buffer(250)
gdf_cemetery_Buffer_250m = gdf_cemetery.copy()

Merge to one geodataframe and dissolve to one geometry

In [ ]:
gdf_exclusion_areas = pd.concat([
    gdf_water_Buffer10m,
    gdf_parks_Buffer10m,
    gdf_transport_Buffer5m,
    gdf_buildings,
    gdf_kindergarten_Buffer_250m,
    gdf_hospital_Buffer_250m,
    gdf_cemetery_Buffer_250m,
], ignore_index=True)

In [29]:
gdf_exclusion_areas.head(50)

,0,geometry,category
0,"POLYGON ((532504.7 5213425.005, 532504.197 521...",None,NaN
1,"POLYGON ((536875.338 5207617.465, 536875.714 5...",None,NaN
2,"POLYGON ((536217.762 5210905.412, 536219.526 5...",None,NaN
3,"POLYGON ((533585.365 5209757.598, 533585.028 5...",None,NaN
4,"POLYGON ((535239.839 5206807.914, 535240.715 5...",None,NaN
5,"POLYGON ((532943.256 5212044.366, 532943.122 5...",None,NaN
6,"POLYGON ((533308.79 5210039.675, 533307.902 52...",None,NaN
7,"POLYGON ((533286.04 5210033.756, 533286.71 521...",None,NaN
8,"POLYGON ((533287.143 5210019.363, 533286.409 5...",None,NaN
9,"POLYGON ((529733.056 5217267.751, 529732.915 5...",None,NaN


In [ ]:
gdf_exclusion_areas.geometry.geom_type.value_counts()

AttributeError: You are calling a geospatial method on the GeoDataFrame, but the active geometry column to use has not been set. 
There are columns with geometry data type ([0, 'geometry']), and you can either set one as the active geometry with df.set_geometry("name") or access the column as a GeoSeries (df["name"]) and call the method directly on it.

Reprojecting DEM from WGS84 to UTM33N and calculate slope 

In [25]:
dem_in = "data/DEM30_Graz.tif"          # original DEM
dem_utm = "data/DEM30_Graz_utm33.tif"   # output DEM (UTM33N)

with rasterio.open(dem_in) as src:
    src_crs = src.crs if src.crs is not None else CRS.from_epsg(4326)
    dst_crs = CRS.from_epsg(32633)  # UTM Zone 33N

    transform, width, height = calculate_default_transform(
        src_crs, dst_crs, src.width, src.height, *src.bounds
    )

    profile = src.profile.copy()
    profile.update({
        "crs": dst_crs,
        "transform": transform,
        "width": width,
        "height": height,
        "nodata": src.nodata if src.nodata is not None else -9999
    })

    with rasterio.open(dem_utm, "w", **profile) as dst:
        dst_array = np.full((height, width), profile["nodata"], dtype=profile["dtype"])
        dst.write(dst_array, 1)

        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=src.transform,
            src_crs=src_crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear,
            src_nodata=src.nodata,
            dst_nodata=profile["nodata"]
        )

In [26]:
dem_path = "data/DEM30_Graz_utm33.tif"
slope_path = "data/dem_slope.tif"

with rasterio.open(dem_path) as src:
    dem = src.read(1, masked=True).astype("float64")
    profile = src.profile.copy()
    transform = src.transform

xres = transform.a
yres = abs(transform.e)

dzdy, dzdx = np.gradient(dem.filled(np.nan), yres, xres)

slope_rad = np.arctan(np.sqrt(dzdx**2 + dzdy**2))
slope_deg = np.degrees(slope_rad)

nodata_out = -9999.0
slope_out = np.where(np.isnan(slope_deg), nodata_out, slope_deg).astype(np.float32)

profile.update(dtype=rasterio.float32, count=1, nodata=nodata_out)

with rasterio.open(slope_path, "w", **profile) as dst:
    dst.write(slope_out, 1)
